In [8]:
import pandas as pd
import altair as alt
from google.colab import files


alt.data_transformers.disable_max_rows()


archivo = "BASE DE DATOS DE CONCIERTOS.editada.xlsx"
df = pd.read_excel(archivo, sheet_name=0, header=0)


df.columns = df.columns.str.strip().str.upper()


df = df[['MERCADO', 'ENTRADA MÁS BARATA', 'ENTRADA MÁS CARA']].dropna()


df['ENTRADA MÁS BARATA'] = pd.to_numeric(df['ENTRADA MÁS BARATA'], errors='coerce')
df['ENTRADA MÁS CARA'] = pd.to_numeric(df['ENTRADA MÁS CARA'], errors='coerce')


df_grouped = df.groupby('MERCADO').agg({
    'ENTRADA MÁS BARATA': 'min',
    'ENTRADA MÁS CARA': 'max'
}).reset_index()


df_long = pd.melt(df_grouped, id_vars='MERCADO',
                  value_vars=['ENTRADA MÁS BARATA', 'ENTRADA MÁS CARA'],
                  var_name='TIPO_ENTRADA', value_name='PRECIO')


df_long['TIPO_ENTRADA'] = df_long['TIPO_ENTRADA'].map({
    'ENTRADA MÁS BARATA': 'Entrada más barata (verde)',
    'ENTRADA MÁS CARA': 'Entrada más cara (morado)'
})


orden_mercados = df_grouped.sort_values('ENTRADA MÁS CARA', ascending=False)['MERCADO'].tolist()
df_long['MERCADO'] = pd.Categorical(df_long['MERCADO'], categories=orden_mercados, ordered=True)


chart = alt.Chart(df_long).mark_line(point=alt.OverlayMarkDef(filled=True)).encode(
    x=alt.X('MERCADO:N', sort=orden_mercados, title=''),
    y=alt.Y('PRECIO:Q', title='Precio en USD'),
    color=alt.Color('TIPO_ENTRADA:N', title='Tipo de entrada',
                    scale=alt.Scale(
                        domain=['Entrada más barata (verde)', 'Entrada más cara (morado)'],
                        range=['green', 'purple']
                    )),
    tooltip=['MERCADO', 'TIPO_ENTRADA', 'PRECIO']
)


text = alt.Chart(df_long).mark_text(
    align='center',
    baseline='bottom',
    dy=-10
).encode(
    x='MERCADO:N',
    y='PRECIO:Q',
    text=alt.Text('PRECIO:Q', format='$,.0f'),
    color=alt.Color('TIPO_ENTRADA:N', legend=None,
                    scale=alt.Scale(
                        domain=['Entrada más barata (verde)', 'Entrada más cara (morado)'],
                        range=['green', 'purple']
                    ))
)


final_chart = (chart + text).properties(
    title=alt.TitleParams(
        text='Precios de entradas por mercado',
        subtitle='Verde: Entrada más barata · Morado: Entrada más cara',
        fontSize=18
    ),
    width=700,
    height=400
)
final_chart.configure_title(fontSize=18)

alt.LayerChart(...)